- Boucle d'entraînement
    - [ ] Définir proprement les métriques d'évaluation
    - [ ] Keep track des métriques d'évaluation
    - [ ] Afficher les métriques d'évaluation
- Mamba
    - [ ] Tester deux configurations de paramètres (celle du papier et une + classique)
    - [ ] Créer un environnement avec mamba
    - [ ] Installer les dépendances avec mamba
- Tester tous les modèles à comparer
    - [ ] Créer un script pour tester tous les modèles à comparer
    - [ ] Créer un script pour tester tous les modèles à comparer avec mamba

In [33]:
import os
import warnings
from time import time
from datetime import datetime
import requests

import pandas as pd
import numpy as np
from PIL import Image

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
from torchvision import transforms
from timm.data.transforms_factory import create_transform

from transformers import AutoModelForImageClassification, AutoImageProcessor, AutoModel

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from constants import ROOT_FOLDER, SEED, VAL_SIZE, TEST_SIZE, BATCH_SIZE, SAMPLING


In [3]:
# Désactiver les alertes de FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

# Print the torch? cuda and cudnn version
print("Torch version: ", torch.__version__)
print("Cuda version: ", torch.version.cuda)
print("CUDNN version: ", torch.backends.cudnn.version())

# Set the device to GPU if available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Torch version:  2.6.0+cu124
Cuda version:  12.4
CUDNN version:  90100
Using device: cuda


[COPY] Récupération des paramètres de train-test-split sur le jeu utilisé

In [4]:
# Assign the folder path containing the images
IMAGE_FOLDER = ROOT_FOLDER / "data" / "images"
IMAGE_TEST = ROOT_FOLDER / "data" / "images" / "2aaa6083689193df5ab01fe37dea1b5e.jpg"
# Assign the folder path containing the former H5 efficientnet weights
ARTIFACTS_FOLDER = ROOT_FOLDER / "artifacts"
# Assign the folder path with the pickle dataset with labels, images filenames and metadata
DATASET_PATH = ROOT_FOLDER / "data" / "dataset_cleaned.pkl"

In [5]:
# Loading the pickle dataset_cleaned used with the previous project as a pandas df
df = pd.read_pickle(DATASET_PATH).drop(columns=['product_name', 'description'])
print(f"Dataset shape: {df.shape}")
print(f"Dataset columns: {df.columns}")

# Encode the labels with LabelEncoder
le = LabelEncoder()
le.fit(df["class"])
n_classes = len(le.classes_)
classes = le.classes_.tolist()
print(f"Number of classes: {n_classes}")
print(f"Classes: {classes}")

# Finally transform the class column to the encoded labels
df["class"] = le.transform(df["class"])

Dataset shape: (1050, 2)
Dataset columns: Index(['image', 'class'], dtype='object')
Number of classes: 7
Classes: ['Baby Care', 'Beauty and Personal Care', 'Computers', 'Home Decor & Festive Needs', 'Home Furnishing', 'Kitchen & Dining', 'Watches']


In [6]:
# Splitting the datasets into train, val and test sets
X_temp, X_test, y_temp, y_test = train_test_split(df['image'], df['class'], test_size=TEST_SIZE, random_state=SEED, stratify=df['class'], shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=VAL_SIZE, random_state=SEED, stratify=y_temp, shuffle=True)

# Concat X and y for each set
train = pd.concat([X_train, y_train], axis=1).sample(SAMPLING) if SAMPLING else pd.concat([X_train, y_train], axis=1)
val = pd.concat([X_val, y_val], axis=1).sample(SAMPLING) if SAMPLING else pd.concat([X_val, y_val], axis=1)
test = pd.concat([X_test, y_test], axis=1).sample(SAMPLING) if SAMPLING else pd.concat([X_test, y_test], axis=1)

# Print the shape of each set
print(f"Train shape: {train.shape}")
print(f"Val shape: {val.shape}")
print(f"Test shape: {test.shape}")

Train shape: (128, 2)
Val shape: (128, 2)
Test shape: (128, 2)


In [7]:
train.head(5)

,image,class
739,52877a6306aef18af67ab54c8233c931.jpg,5
366,47d7792e50e69b048b1f17176f170141.jpg,1
687,d61b368146f83075937e144dab93c6a1.jpg,1
305,906b5b16c78ba1c718501138702cb32c.jpg,3
610,2414e4b7e5948263b8dbb4843557be4d.jpg,3


In [8]:
val.head(5)

,image,class
330,394c2c627914e1eed9b8ac343583a679.jpg,2
465,da2ce8d17a17f9d19b368434aedea5e9.jpg,0
401,50be4cd92fda2755e1e884421a52e345.jpg,1
304,95cb9561009bd6707c67f05e6a00c16e.jpg,3
596,e4922f01eda047582cd72e9d1063ab7a.jpg,3


In [9]:
test.head(5)

,image,class
1046,fd6cbcc22efb6b761bd564c28928483c.jpg,0
805,0b793c1727ef52285a25dedf8b86626b.jpg,1
480,88ca277594f22bb7494f2d0bbc51f8ba.jpg,2
1049,f2f027ad6a6df617c9f125173da71e44.jpg,0
591,62a1044ba64031b9e40f98a4fb890d9d.jpg,4


In [10]:
# Block DataLoader
class ImageDataset(Dataset):
    def __init__(self, dataframe, image_dir, processor=None, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): DataFrame containing image file names and labels.
            image_dir (str): Directory where images are stored.
            processor (AutoImageProcessor, optional):  Hugging Face processor for image preprocessing. Defaults to None.
            transform (callable, optional): Optional transform to be applied on a sample. Defaults to None.
        """
        super().__init__()
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.processor = processor
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_dir, self.dataframe.iloc[idx, 0])  # Assuming image file names are in the first column
        image = Image.open(img_name).convert('RGB')  # Ensure consistent color format

        label = self.dataframe.iloc[idx, 1]  # Assuming labels are in the second column

        if self.processor:
            inputs = self.processor(images=image, return_tensors="pt")
            image = inputs['pixel_values'].squeeze()  # Remove batch dimension
        elif self.transform:
            image = self.transform(image)

        # Convert label to tensor
        label = torch.tensor(label)

        return image, label

# **EXPERIMENTS**

In [56]:
def classification_report_to_markdown(y_true, y_pred, target_names):
    """
    Convert the classification report to a markdown table.
    """
    report = classification_report(y_true, y_pred, target_names=target_names, zero_division=0, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_df = report_df.drop(columns=['support'])
    report_df = report_df.rename_axis('Classes').reset_index()
    return report_df.to_markdown(index=False)

In [55]:
def test_model(model: AutoModelForImageClassification, experiment_id: str, test_loader: DataLoader, criterion: nn.Module, writer: SummaryWriter = None):
    # Load the best model
    model.load_state_dict(torch.load(ARTIFACTS_FOLDER / f'{experiment_id}.pth'))
    model.to(DEVICE)
    model.eval()

    # Initialize the test metrics
    test_loss, correct_test, total_test = .0, 0, 0
    running_test_steps = 0
    running_test_time_by_step = .0
    y_true, y_pred = [], []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            step_time = time()
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(inputs).logits
            test_loss += criterion(outputs, labels).item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            correct_test += (preds == labels).sum().item()
            total_test += inputs.size(0)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            running_test_time_by_step += time() - step_time
            running_test_steps += 1
            if writer:
                writer.add_scalar('TimingByStep/test', running_test_time_by_step, running_test_steps)

    epoch_test_loss = test_loss / total_test
    epoch_test_acc  = correct_test / total_test
    if writer:
        writer.add_scalar('Accuracy/test', epoch_test_acc, 0)

    print(f"Test Loss: {epoch_test_loss:.4f}, Test Acc: {epoch_test_acc:.4f}")
    
    # Save Classification report in tensorboard
    classification_report_md = classification_report_to_markdown(y_true, y_pred, target_names=classes)
    cm = ConfusionMatrixDisplay.from_predictions(y_true, y_pred, labels=range(n_classes), normalize='true', display_labels=classes, xticks_rotation="vertical", cmap=plt.cm.Blues)
    # Save the classification report as a markdown file in tensorboard
    if writer:
        writer.add_text('ClassificationReport/test', classification_report_md, 0)
        writer.add_figure('ConfusionMatrix/test', cm.figure_, 0)
    # Delete the model from GPU memory
    del model, inputs, labels, outputs
    torch.cuda.empty_cache()

In [57]:
def train_and_eval_amfic(
    model: AutoModelForImageClassification,
    model_card: str,
    train_loader: DataLoader,
    val_loader: DataLoader,
    test_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: nn.Module,
    num_epochs: int,):
    # --- LOOP ---
    # Initialize SummaryWriter
    experiment_id = "_".join([datetime.now().strftime("%Y%m%d-%H%M%S"), model_card.split("/")[-1]])
    log_dir = ROOT_FOLDER / os.getcwd().split("/")[-1] / "runs" / "_".join([experiment_id, model_card.split("/")[-1]])
    writer = SummaryWriter(log_dir)

    # Initialize commun metrics
    best_epoch = 0
    
    # Initialize the training metrics
    running_train_time_by_step = .0
    running_train_time_by_epoch = .0
    running_train_steps = 0
    writer.add_scalar('TimingByEpoch/train', running_train_time_by_epoch, 0)

    # Initialize the validation metrics
    best_val_metric = float('-inf')
    running_val_time_by_step = .0
    running_val_time_by_epoch = .0
    running_val_steps = 0
    writer.add_scalar('TimingByEpoch/validation', running_val_time_by_epoch, 0)
    
    writer.add_scalar('TimingByEpoch/train', running_train_time_by_epoch, 0)
    # Move model to the device
    model.to(DEVICE)

    # Training loop
    for epoch in range(num_epochs):
        # --- 1. TRAINING LOOP ---
        model.train()
        train_loss, correct_train, total_train = 0.0, 0, 0
        epoch_time = time()
        for i, (inputs, labels) in enumerate(train_loader): # Use enumerate for step count
            step_time = time()
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            preds_train = outputs.argmax(dim=1)
            correct_train += (preds_train == labels).sum().item()
            total_train += inputs.size(0)
            running_train_steps += 1
            running_train_time_by_step += time() - step_time
            writer.add_scalar('TimingByStep/train', running_train_time_by_step, running_train_steps)

        running_train_time_by_epoch += time() - epoch_time
        epoch_train_loss = train_loss / total_train
        epoch_train_acc = correct_train / total_train
        # Log training metrics per epoch
        writer.add_scalar('Loss/train', epoch_train_loss, epoch)
        writer.add_scalar('Accuracy/train', epoch_train_acc, epoch)
        writer.add_scalar('TimingByEpoch/train', running_train_time_by_epoch, epoch)
        print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}")


        # --- 2. VALIDATION LOOP ---
        model.eval()
        val_loss, correct_val, total_val = .0, 0, 0
        epoch_time = time()
        with torch.no_grad():
            for inputs, labels in val_loader:
                # Move images and labels to the device
                step_time = time()
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
                outputs = model(inputs).logits
                val_loss   += criterion(outputs, labels).item() * inputs.size(0)
                preds      = outputs.argmax(dim=1)
                correct_val   += (preds == labels).sum().item()
                total_val     += inputs.size(0)
                running_val_steps += 1
                running_val_time_by_step += time() - step_time
                writer.add_scalar('TimingByStep/validation', running_val_time_by_step, running_val_steps)

        running_val_time_by_epoch += time() - epoch_time
        epoch_val_loss = val_loss / total_val
        epoch_val_acc  = correct_val / total_val
        # Log validation metrics per epoch
        writer.add_scalar('Loss/validation', epoch_val_loss, epoch)
        writer.add_scalar('Accuracy/validation', epoch_val_acc, epoch)
        writer.add_scalar('TimingByEpoch/validation', running_val_time_by_epoch, epoch)
        print(f"Epoch [{epoch}/{num_epochs}], Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

        # --- 3. UPDATE BEST MODEL ---
        # Save the model if the validation accuracy is better than the best one
        if epoch_val_acc > best_val_metric:
            best_val_metric = epoch_val_acc
            best_epoch      = epoch
            torch.save(model.state_dict(), ARTIFACTS_FOLDER / f'{experiment_id}.pth')
            print(f"Best model updated at epoch {best_epoch} with val acc: {best_val_metric:.4f}")
    
    # Delete the model from GPU memory
    del inputs, labels, outputs, optimizer, loss
    torch.cuda.empty_cache()

    # --- 4. TESTING LOOP ---
    test_model(model, experiment_id, test_loader, criterion, writer)

    # --- After the training loop ---
    writer.close() # Close the writer

## **Google VIT**

In [58]:
# Assigne the model card name
model_card = "google/vit-base-patch16-224-in21k"

# Define the model
model = AutoModelForImageClassification.from_pretrained(model_card, num_labels=n_classes, trust_remote_code=True)

# Define the image processor
processor = AutoImageProcessor.from_pretrained(model_card)

# Prepare the dataloaders for training, validation and testing
dataset = ImageDataset(dataframe=train, image_dir=IMAGE_FOLDER, processor=processor)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
dataset = ImageDataset(dataframe=val, image_dir=IMAGE_FOLDER, processor=processor)
val_loader = DataLoader(dataset, batch_size=BATCH_SIZE)
dataset = ImageDataset(dataframe=test, image_dir=IMAGE_FOLDER, processor=processor)
test_loader = DataLoader(dataset, batch_size=BATCH_SIZE)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [59]:
# TODO : Ajouter une card dans le tensorboard pour les paramètres du modèle
# TODO : warning PIL decompression bomb warning
# TODO : Nettoyer les fichiers useless

# Set the training parameters
num_epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Run the training workflow for AutoModelForImageClassification
train_and_eval_amfic(
    model = model,
    model_card = model_card,
    train_loader = train_loader,
    val_loader = val_loader,
    test_loader = test_loader,
    optimizer = optimizer,
    criterion = criterion,
    num_epochs = num_epochs,)

Epoch [0/5], Train Loss: 1.9501, Train Acc: 0.1016
Epoch [0/5], Val Loss: 1.9016, Val Acc: 0.2891
Best model updated at epoch 0 with val acc: 0.2891
Epoch [1/5], Train Loss: 1.8419, Train Acc: 0.4844
Epoch [1/5], Val Loss: 1.8628, Val Acc: 0.4141
Best model updated at epoch 1 with val acc: 0.4141
Epoch [2/5], Train Loss: 1.7522, Train Acc: 0.7500
Epoch [2/5], Val Loss: 1.8219, Val Acc: 0.5312
Best model updated at epoch 2 with val acc: 0.5312
Epoch [3/5], Train Loss: 1.6580, Train Acc: 0.8750
Epoch [3/5], Val Loss: 1.7769, Val Acc: 0.6094
Best model updated at epoch 3 with val acc: 0.6094
Epoch [4/5], Train Loss: 1.5608, Train Acc: 0.9531
Epoch [4/5], Val Loss: 1.7335, Val Acc: 0.6328
Best model updated at epoch 4 with val acc: 0.6328
Test Loss: 1.7634, Test Acc: 0.5547


## MOBILENETV2

In [ ]:
"google/mobilenet_v2_1.0_224"

## **MAMBA S 1 K**

In [17]:
model = AutoModel.from_pretrained("nvidia/MambaVision-S-1K", trust_remote_code=True)

# eval mode for inference
model.cuda().eval()

# prepare image for the model
url = 'http://images.cocodataset.org/val2017/000000020247.jpg'
image = Image.open(requests.get(url, stream=True).raw)
input_resolution = (3, 224, 224)  # MambaVision supports any input resolutions

transform = create_transform(input_size=input_resolution,
                             is_training=False,
                             mean=model.config.mean,
                             std=model.config.std,
                             crop_mode=model.config.crop_mode,
                             crop_pct=model.config.crop_pct)
inputs = transform(image).unsqueeze(0).cuda()
# model inference
out_avg_pool, features = model(inputs)
print("Size of the averaged pool features:", out_avg_pool.size())  # torch.Size([1, 768])
print("Number of stages in extracted features:", len(features)) # 4 stages
print("Size of extracted features in stage 1:", features[0].size()) # torch.Size([1, 96, 56, 56])
print("Size of extracted features in stage 4:", features[3].size()) # torch.Size([1, 768, 7, 7])

Size of the averaged pool features: torch.Size([1, 768])
Number of stages in extracted features: 4
Size of extracted features in stage 1: torch.Size([1, 96, 56, 56])
Size of extracted features in stage 4: torch.Size([1, 768, 7, 7])


In [18]:
num_classes = 7

class MambaClassifier(nn.Module):
    def __init__(self, backbone, num_classes, hidden_dim=768): # Added hidden_dim
        super().__init__()
        self.backbone = backbone
        self.config = self.backbone.config
        # self.classifier = nn.Linear(640, num_classes) # Original classifier
        self.classifier = nn.Linear(hidden_dim, num_classes) # Modified classifier

    def forward(self, x):
        out_avg_pool, _ = self.backbone(x)
        logits = self.classifier(out_avg_pool)
        return logits


In [19]:
mamba = MambaClassifier(model, num_classes=num_classes).cuda().eval()

image = Image.open(img_test)

input_resolution = (3, 224, 224)  # MambaVision supports any input resolutions

transform = create_transform(input_size=input_resolution,
                             is_training=False,
                             mean=mamba.config.mean,
                             std=mamba.config.std,
                             crop_mode=mamba.config.crop_mode,
                             crop_pct=mamba.config.crop_pct)


inputs = transform(image).unsqueeze(0).cuda()
# Prédiction
with torch.no_grad():
    logits = mamba(inputs)
    probs = torch.softmax(logits, dim=-1)

print("Probabilités prédites:", probs)

Probabilités prédites: tensor([[0.1360, 0.1641, 0.1267, 0.1375, 0.1245, 0.1609, 0.1504]],
       device='cuda:0')


In [20]:
model = MambaClassifier(AutoModel.from_pretrained("nvidia/MambaVision-S-1K", trust_remote_code=True), num_classes=num_classes).cuda().train()

image = Image.open(img_test)

input_resolution = (3, 224, 224)  # MambaVision supports any input resolutions

transform = create_transform(input_size=input_resolution,
                             is_training=True,
                             mean=mamba.config.mean,
                             std=mamba.config.std,
                             crop_mode=mamba.config.crop_mode,
                             crop_pct=mamba.config.crop_pct)

dataset = ImageDataset(dataframe=train, image_dir=image_dir, transform=transform)

# 5. Create DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [21]:
# Training loop for the model with the train dataloader
# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move model to the device
model.to(device)
# Set the model to training mode
model.train()
# Set the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
# Set the number of epochs
num_epochs = 3
# Set the loss function
criterion = torch.nn.CrossEntropyLoss()
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        
        # inputs = transform(image).unsqueeze(0).cuda()
        # model inference
        # outputs = model(inputs)
        # end_time = time()
        # logits = outputs["logits"]
        # predicted_class_idx = logits.argmax(-1).item()

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}")
# Save the model
# model.save_pretrained("mamba_model")
# processor.save_pretrained("mamba_processor")

Epoch [1/3], Loss: 1.9875
Epoch [2/3], Loss: 1.9445
Epoch [3/3], Loss: 1.9724


In [22]:
# Inference on test set
# Set the model to evaluation mode
model.eval()
# Initialize the test dataset
test_dataset = ImageDataset(dataframe=test, image_dir=image_dir, transform=transform)
# Initialize the test dataloader
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)
# Initialize the predictions list
predictions = []
# Initialize the labels list
labels_list = []
# Inference loop
for images, labels in test_dataloader:
    # Move images to the device
    images = images.to(device)
    # Forward pass
    with torch.no_grad():
        outputs = model(images)
    # Get the predicted labels
    _, preds = torch.max(outputs, 1)
    # Append the predictions and labels to the lists
    predictions.extend(preds.cpu().numpy())
    labels_list.extend(labels.cpu().numpy())
# Convert the predictions and labels to numpy arrays
predictions = np.array(predictions)
labels_list = np.array(labels_list)
# Print the classification report
print(classification_report(labels_list, predictions, target_names=classes))
# Save the predictions

                            precision    recall  f1-score   support

           Home Furnishing       0.00      0.00      0.00         4
                 Baby Care       0.00      0.00      0.00         5
                   Watches       0.00      0.00      0.00         4
Home Decor & Festive Needs       0.00      0.00      0.00         6
          Kitchen & Dining       0.40      0.40      0.40         5
  Beauty and Personal Care       0.10      0.20      0.13         5
                 Computers       0.00      0.00      0.00         3

                  accuracy                           0.09        32
                 macro avg       0.07      0.09      0.08        32
              weighted avg       0.08      0.09      0.08        32



## **MAMBA B 21 K**

In [23]:
model = MambaClassifier(AutoModel.from_pretrained("nvidia/MambaVision-B-21K", trust_remote_code=True), num_classes=num_classes, hidden_dim=1024).cuda().train()

image = Image.open(img_test)

input_resolution = (3, 224, 224)  # MambaVision supports any input resolutions

transform = create_transform(input_size=input_resolution,
                             is_training=True,
                             mean=mamba.config.mean,
                             std=mamba.config.std,
                             crop_mode=mamba.config.crop_mode,
                             crop_pct=mamba.config.crop_pct)

dataset = ImageDataset(dataframe=train, image_dir=image_dir, transform=transform)

# 5. Create DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [24]:
# Training loop for the model with the train dataloader
# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move model to the device
model.to(device)
# Set the model to training mode
model.train()
# Set the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
# Set the number of epochs
num_epochs = 3
# Set the loss function
criterion = torch.nn.CrossEntropyLoss()
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        
        # inputs = transform(image).unsqueeze(0).cuda()
        # model inference
        # outputs = model(inputs)
        # end_time = time()
        # logits = outputs["logits"]
        # predicted_class_idx = logits.argmax(-1).item()

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}")
# Save the model
# model.save_pretrained("mamba_model")
# processor.save_pretrained("mamba_processor")

Epoch [1/3], Loss: 2.1026
Epoch [2/3], Loss: 2.0404
Epoch [3/3], Loss: 1.9177


In [25]:
# Inference on test set
# Set the model to evaluation mode
model.eval()
# Initialize the test dataset
test_dataset = ImageDataset(dataframe=test, image_dir=image_dir, transform=transform)
# Initialize the test dataloader
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)
# Initialize the predictions list
predictions = []
# Initialize the labels list
labels_list = []
# Inference loop
for images, labels in test_dataloader:
    # Move images to the device
    images = images.to(device)
    # Forward pass
    with torch.no_grad():
        outputs = model(images)
    # Get the predicted labels
    _, preds = torch.max(outputs, 1)
    # Append the predictions and labels to the lists
    predictions.extend(preds.cpu().numpy())
    labels_list.extend(labels.cpu().numpy())
# Convert the predictions and labels to numpy arrays
predictions = np.array(predictions)
labels_list = np.array(labels_list)
# Print the classification report
print(classification_report(labels_list, predictions, target_names=classes))
# Save the predictions

                            precision    recall  f1-score   support

           Home Furnishing       0.00      0.00      0.00         4
                 Baby Care       0.50      0.40      0.44         5
                   Watches       0.17      0.25      0.20         4
Home Decor & Festive Needs       0.20      0.17      0.18         6
          Kitchen & Dining       0.25      0.20      0.22         5
  Beauty and Personal Care       0.25      0.20      0.22         5
                 Computers       0.00      0.00      0.00         3

                  accuracy                           0.19        32
                 macro avg       0.20      0.17      0.18        32
              weighted avg       0.21      0.19      0.20        32

